In [1]:
import pandas as pd
import seaborn as sb
import numpy as np

In [73]:
import os
import re
from collections import Counter

global control_var
control_var = True
def get_tuneBooks_file_names(path):
    tuneBook_filenames = []
    
    for tuneBook_name in os.listdir(path):
        if tuneBook_name.endswith('.abc'):
            tuneBook_filenames.append(tuneBook_name)
    
    #return ["jigs.abc"]
    return tuneBook_filenames
def split_abc_song(abc_song):
    k_index = abc_song.find('K:')

    split_index = abc_song.find('\n', k_index)

    header = abc_song[:split_index]
    body = abc_song[split_index+1:]
    return header, body

def tuneBook_to_dataframe(tuneBook):
    song_list = contents.split("\n\n")
    song_list = [song.strip() for song in song_list]
    songs_header_body_format = [split_abc_song(song) for song in song_list if song != '']
    """
    for (h,b) in songs_header_body_format:
        print("header")
        print(h)
        print("*"*10)
        print("body")
        print(b)
        print("\n")
        print("-"*50)
        print("\n")
    """
    
    return pd.DataFrame(songs_header_body_format, columns=['header', 'body'])

def get_attributes_from_song_header(tuneBook_df,tuneBook_name):
    attributes_column = tuneBook_df["header"]
    attributes_list = list(attributes_column)
    
    data = []
    for string in attributes_list:
        song = {}
        for line in string.split("\n"):
            if ':' in line:
                key, value = line.split(":", maxsplit=1)
                song[key.strip()] = value.strip()
        data.append(song)

    df = pd.DataFrame(data)
    descriptive_names = {
                        'X': 'reference_number',
                        'T': 'title',
                        'S': 'source',
                        'M': 'meter',
                        'L': 'unit_note_length',
                        'R': 'rhythm',
                        'P': 'parts',
                        'K': 'key',
                        'F': 'file_name',
                        'N': 'notes'
                        }
    df.rename(columns=descriptive_names, inplace=True)
    df["tuneBook"] = tuneBook_name
    df["original_header"] =  tuneBook_df["header"]
    df["original_body"] =  tuneBook_df["body"]
    
    return df

def get_chord_progression(song_body):
    bars = song_body.split("|")
    chord_progression = "|"
    for bar_content in bars:
        match_chords = r'"[^"]*"'
        chords = re.findall(match_chords, bar_content)
        bar_chords = ''.join(chords)
        if bar_chords != "":
            chord_progression += bar_chords +'|'
    return chord_progression

def get_chord_occurrences(song_body):
    bars = song_body.split("|")
    chord_progression = "|"
    match_chords = r'"[^"]*"'
    chords = [re.findall(match_chords, bar_content)  for bar_content in bars if bar_content]
    chords = [chord for bar_chords in chords for chord in bar_chords]
    chord_occurrences = dict(Counter(chords))
    return chord_occurrences

def get_chords_data(song_body=""):
    song_body = 'P:A\nD|"Gm"GB2G|"D7"^Fd2D|"Gm"GB2G|"D7"^FA2D|"Gm"GB2G|"D"^Fg -"Cm"gc|\\\n"Gm/d"B2 "D7"A2|"Gm"G3:|\nP:B\nd|"Gm"dg2d|"Gm"gb2d|"F"cf2c|"F"fa2c|"Eb"Be2d/2c/2|"Gm"dg -"Cm"gc|\\\n"Gm/d"B2 "D7"A2|"Gm"G3:|'
    clean_body_song = ""
    match_pattern = ""
    for line in song_body.splitlines():
        if not "P:" in line:
            new_line = line.strip("\\")
            match_pattern = r"[: ]"
            new_line = re.sub(match_pattern, "", new_line)
            clean_body_song = clean_body_song+new_line

    
    chord_progression = get_chord_progression(clean_body_song)
    chord_occurrences = get_chord_occurrences(clean_body_song)
    return chord_progression,chord_occurrences

### treat strings line by line to ignore comments and parts!!!!!
def get_song_characteristics_from_body(tuneBook_df, header_col_name="original_body"):
    songs_bodies=tuneBook_df[header_col_name]
    global control_var
    print(songs_bodies)
    if(control_var):
        control_var = False
        progression,occurrences = get_chords_data()
        print(progression)
        print(occurrences)
    # get anacrusis bool to see that the dataset is balanced
    #tuneBook["num_notes_per_bar"] # count num -> useless? to measure intensity?
    #tuneBook["num_bars"] # count number of || ins song -> to detect possible outliers?
    #tuneBook["num_notes_in_song"] # count a,b,c,d...in string, match regex "a"   
    #tuneBook["chord_progression"] #string "|"C"|"Dm"|" -> to count chords and to have a "Tree view of common chord progressions"
    #tuneBook["multiple_parts"] #bool -> to treat songs with multiple parts
    ##  Count types of notes in each song to see the proportion of notes for a given key
    ## Maybe by chord also?
    return []
def get_songs_metadata(songs_dataframe, tuneBook_name):
    #songs_dataframe["number_in_tunebook"] = get_attributes_from_song_header(songs_dataframe)
    df = get_attributes_from_song_header(songs_dataframe,tuneBook_name)
    get_song_characteristics_from_body(df)
    return df

absolute_path = os.getcwd()
relative_path = "notebooks/data/NottinghamData/nottingham_database"
absolute_path = os.path.join(absolute_path, relative_path)
songs_df = pd.DataFrame()
list_tuneBooks = get_tuneBooks_file_names(absolute_path)
for abc_tuneBook_filename in list_tuneBooks:
    file_path = os.path.join(absolute_path, abc_tuneBook_filename)
    
    with open(file_path) as tuneBook:
        contents = tuneBook.read()
        df = tuneBook_to_dataframe(contents)
        df1 = get_songs_metadata(df,abc_tuneBook_filename)
        songs_df = pd.concat([songs_df, df1], ignore_index=True)

#print(songs_df.isna().sum())
#songs_df["original_body"][0]

0     P:A\nD|"Gm"GB2G|"D7"^Fd2D|"Gm"GB2G|"D7"^FA2D|"...
1     P:A\n|G2E |"G"D3 -D2E "f#"G2B|"Em"A3 G3 -"d"G2...
2     D2G |"G"B3 -B3 BAG|"G"dB2 -B3 ABd|"G"d3 -d2B "...
3     %%MIDI gchord fzczcz\n"G"g2d4B2-"D7"B2A2|"G"Bc...
4     %%MIDI gchord fzczcz\nP:A\nB2c2||\\nP:segno\n"...
5     P:A\nf/2g/2|"D"af df|"A7"e2 A2|"D"FA Af|"Em"e2...
6     %%MIDI gchord fzczcz\nP:A\n"Gm"g4"D7"^f2=ef"Gm...
7     P:A\n|:D|"G"GB BA/2B/2|"D7"cA Ad|"G"BG GB|"Am"...
8     P:A\n"G"DED G2A|"G"BcB A2G|"G7"dB2 A2G|"C"E3 -...
9     a|"Dm"af ga|"Dm"fe/2f/2 da|"Dm"af ga|"Dm"f2 fa...
10    P:A\nA|"D"d2e fdf|"G"g3 "A7"fga|"D"fgf "Em"e2d...
11    A|"D"d2 "A7"de|"D"f2 ef|"G"gg "A7"fe|"D"d3:|\n...
12    %%MIDI gchord fzzczz\nP:A\n^c|"Dm"dAf "A7"eAg|...
13    "D"dd "A"ce|"G"BB "A"A3/2A/2|"Em"BB "A7"cA|"D"...
14    |:"Dm"Ad "A"=B^c|"Gm"dc/2B/2 "Dm"A3/2B/2|"Dm"c...
Name: original_body, dtype: object
|"Gm"|"D7"|"Gm"|"D7"|"Gm"|"D""Cm"|"Gm/d""D7"|"Gm"|"Gm"|"Gm"|"F"|"F"|"Eb"|"Gm""Cm"|"Gm/d""D7"|"Gm"|
{'"Gm"': 8, '"D7"': 4, '"D

In [9]:
songs_df["original_body"].str.contains("P:")

0        True
1        True
2       False
3       False
4        True
        ...  
1033    False
1034     True
1035    False
1036     True
1037     True
Name: original_body, Length: 1038, dtype: bool

|"Gm"|"D7"|"Gm"|"D7"|"Gm"|"D""Cm"|"Gm/d""D7"|


'|"Gm"|"D7"|"Gm"|"D7"|"Gm"|"D""Cm"|"Gm/d""D7"|'